# Imports

In [4]:
# !pip install a2pm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time

In [2]:
from a2pm import A2PMethod
from a2pm.callbacks import BaseCallback, MetricCallback, TimeCallback
from a2pm.patterns import BasePattern, CombinationPattern, IntervalPattern
from a2pm.wrappers import BaseWrapper, KerasWrapper, SklearnWrapper, TorchWrapper

In [3]:
# X_train = np.load('X-IIoT-pre-processed/x_train.npy')
# y_train = np.load('X-IIoT-pre-processed/y_train.npy')
# X_val = np.load('X-IIoT-pre-processed/x_val.npy')
# y_val = np.load('X-IIoT-pre-processed/y_val.npy')
# X_test = np.load('X-IIoT-pre-processed/x_test.npy')
# y_test = np.load('X-IIoT-pre-processed/y_test.npy')

X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

# Train

We train the baseline model to get an impression of how our model perform on an IDS dataset without any adversarial samples.

In [4]:
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV

In [5]:
model_xgb = XGBClassifier()

param_grid = {
    # 'n_estimators': [10, 50, 100],
    # 'max_depth': [None, 50, 100],
    # 'n_estimators': [10],
    # 'max_depth': [None],
}

grid_search_xgb = GridSearchCV(model_xgb, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_xgb.fit(X_train, y_train)

print("Best parameters found: ", grid_search_xgb.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   2.6s
[CV 2/3] END ..................................., score=1.000 total time=   2.6s
[CV 3/3] END ..................................., score=1.000 total time=   2.4s
Best parameters found:  {}


# Evaluate

Evaluate the RF model on the IDS dataset, once again with no adversarial samples.

## Helpers

In [6]:


def metrics_master(model, X_test, y_test, type):
    metrics_weighted(model, X_test, y_test, type)
    metrics_macro(model, X_test, y_test, type)

def metrics_macro(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    data = {
    "Metric": ["Accuracy", "Macro Precision", "Macro Recall", "Macro F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/XGB.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Macro Precision: {precision * 100:.2f}%")
    print(f"Macro Recall: {recall * 100:.2f}%")
    print(f"Macro F1 Score: {f1 * 100:.2f}%")
    
def metrics_weighted(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    data = {
    "Metric": ["Accuracy", "Weighted Precision", "Weighted Recall", "Weighted F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/XGB.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Weighted Precision: {precision * 100:.2f}%")
    print(f"Weighted Recall: {recall * 100:.2f}%")
    print(f"Weighted F1 Score: {f1 * 100:.2f}%")

## Scores

In [7]:
xgb_model = grid_search_xgb.best_estimator_
metrics_master(xgb_model, X_test, y_test, "original score")

original score
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
original score
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%


# A2PM Generation

Generate A2PM adversarial samples for further experiments

Since we know the data is pre-processed, the categorical variables will be binary columns, and conversely the numerical variables will be the others. Let's find these columns.

In [8]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

In [9]:
X_train.shape

(716256, 42)

The following configuration is the boilerplate found on the A2PM github (https://github.com/vitorinojoao/a2pm). The only modifications are that the interval pattern is applied to the numerical columns, and the combination pattern is applied to the categorical (binary) variables. I removed integer features since the numerical columns are float values, and removed locked_features as I wasn't sure which features needed to be kept static (data was given as processed .npy files)

In [10]:
xgb_base = grid_search_xgb.best_estimator_

classifier = SklearnWrapper(xgb_base)


# rule of thumb: Interval for numerical, combination for categorical
pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 1,
    },
    # {
    #     "type": "combination",
    #     "features": binary_columns,
    #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
    #     "probability": 0.4,
    # },
)

method = A2PMethod(pattern)

We will generate attacks based off the entire dataset. We will resplit the train-test sets afterward in our experiments

In [11]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [12]:
start_time = time.time()

X_adversarial = method.fit_generate(classifier, X, y)

training_time = time.time() - start_time
print(f"Attack Time: {training_time}")

Attack Time: 541.8455910682678


# Scores after A2PM Attack

Here we are evaluating how our trained (unprotected) RF model performs on the full A2PM dataset. Each sample is now perturbed, but should still correspond to the original target classes y

In [13]:
metrics_master(xgb_model, X_adversarial, y, "full A2PM Attack")

full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
full A2PM Attack
Accuracy: 99.99%
Macro Precision: 98.04%
Macro Recall: 92.18%
Macro F1 Score: 94.63%


# Adversarial Training Preprocess

As per many adversarial defense papers, adversarial training has been cited as one of the best ways to improve resilience. We will retrain our model with the original data, in addition to an added 10% of the data being adversarial samples with an "adversarial" class (class 19).

In [14]:
X_train.shape[0]+X_test.shape[0]

955009

In [15]:

adv_samples_num = int(955009*0.10/0.9) 
adv_samples_num # 10%

106112

In [16]:
X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
X_adversarial_sampled = X_adversarial[X_adversarial_indices]

y_adv = np.full(adv_samples_num, 5)
y_adv

array([5, 5, 5, ..., 5, 5, 5])

In [17]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [18]:
X_new = X.copy()
y_new = y.copy()

X_combined = np.vstack((X_new, X_adversarial_sampled))
y_combined = np.concatenate((y_new, y_adv))

# Adversarial Training

Our whole data now contains 10% A2PM samples. Assuming a uniformly random 80/20 split, our training data will be expected to contain 8% A2PM samples, while the testing data is expected to contain 2% A2PM samples. 

In [19]:
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

In [20]:
model_xgb_protected = XGBClassifier()

param_grid = {
    # 'n_estimators': [10, 50, 100],
    # 'max_depth': [None, 50, 100],
    # 'n_estimators': [10],
    # 'max_depth': [None],
}

grid_search_xgb_protected = GridSearchCV(model_xgb_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_xgb_protected.fit(X_train_adv, y_train_adv)

print("Best parameters found: ", grid_search_xgb.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   4.0s
[CV 2/3] END ..................................., score=1.000 total time=   4.0s
[CV 3/3] END ..................................., score=1.000 total time=   4.2s
Best parameters found:  {}


In [21]:
xgb_model_protected = grid_search_xgb_protected.best_estimator_

# Scores (Binary A2PM Detection after Adversarial Training)

We want to see how well our protected model can detect A2PM samples. We run it against the unseen A2PM set

In [32]:
X_adversarial_unseen = X_adversarial[np.logical_not(np.isin(np.arange(X_adversarial.shape[0]), X_adversarial_indices))]
len(X_adversarial_unseen)

848897

In [33]:
protected_adversarial_preds = xgb_model_protected.predict(X_adversarial_unseen)

All we need to know is if the model correctly predicts every sample as class 19 (A2PM), and as such we will treat this as a binary classification problem.

In [34]:
y_adv_full = np.full(len(X_adversarial_unseen), 5)

positive_class = 5
y_test_binary = (y_adv_full == positive_class).astype(int)
adversarial_preds_binary = (protected_adversarial_preds == positive_class).astype(int)

In [35]:
adversarial_accuracy = accuracy_score(y_test_binary, adversarial_preds_binary)
adversarial_precision = precision_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_recall = recall_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_f1 = f1_score(y_test_binary, adversarial_preds_binary, average='binary')

print(f"Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%")
print(f"Adversarial Precision: {adversarial_precision * 100:.2f}%")
print(f"Adversarial Recall: {adversarial_recall * 100:.2f}%")
print(f"Adversarial F1 Score: {adversarial_f1 * 100:.2f}%")

Adversarial Accuracy: 100.00%
Adversarial Precision: 100.00%
Adversarial Recall: 100.00%
Adversarial F1 Score: 100.00%


# Scores (10% A2PM Samples, no Adv Training)

Here we benchmark how well our unprotected model performs on a test set with 10% A2PM samples and 90% real samples

In [22]:
metrics_master(xgb_model, X_test_adv, y_test_adv, "no Adv Training")

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


no Adv Training
Accuracy: 89.95%
Weighted Precision: 80.91%
Weighted Recall: 89.95%
Weighted F1 Score: 85.19%
no Adv Training
Accuracy: 89.95%
Macro Precision: 75.38%
Macro Recall: 83.33%
Macro F1 Score: 79.12%


# Scores (10% A2PM Samples w/ Adv Training)

Here we benchmark how well our protected model performs on a test set with 10% A2PM samples and 90% real samples

In [23]:
metrics_master(xgb_model_protected, X_test_adv, y_test_adv, "w/ Adv Training")

w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
w/ Adv Training
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%


In [54]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

xgb_base = grid_search_xgb.best_estimator_

classifier = SklearnWrapper(xgb_base)


ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    pattern = (
        {
            "type": "interval",
            "features": numerical_columns,
            #"integer_features": list(range(10, 20)),
            "ratio": ratio,
            "max_ratio": 0.3,
            "missing_value": 0.0,
            "probability": 1,
        },
        # {
        #     "type": "combination",
        #     "features": binary_columns,
        #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
        #     "probability": 0.4,
        # },
    )
    
    method = A2PMethod(pattern)
    
    X = np.concatenate((X_train, X_test), axis=0)
    y = np.concatenate((y_train, y_test), axis=0)
    
    start_time = time.time()
    
    X_adversarial = method.fit_generate(classifier, X, y)
    
    training_time = time.time() - start_time
    print(f"Attack Time: {training_time}")

    text = f"{ratio}:full A2PM Attack"
    metrics_master(xgb_model, X_adversarial, y, text)
    
    
    adv_samples_num = int(955009*0.10/0.9) 
    
    X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
    X_adversarial_sampled = X_adversarial[X_adversarial_indices]
    
    y_adv = np.full(adv_samples_num, 5)
    
    X_new = X.copy()
    y_new = y.copy()
    
    X_combined = np.vstack((X_new, X_adversarial_sampled))
    y_combined = np.concatenate((y_new, y_adv))
    
    X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)
    
    model_xgb_protected = XGBClassifier()
    
    param_grid = {
        # 'n_estimators': [10, 50, 100],
        # 'max_depth': [None, 50, 100],
        # 'n_estimators': [10],
        # 'max_depth': [None],
    }
    
    grid_search_xgb_protected = GridSearchCV(model_xgb_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
    grid_search_xgb_protected.fit(X_train_adv, y_train_adv)
    
    print("Best parameters found: ", grid_search_xgb.best_params_)
    
    xgb_model_protected = grid_search_xgb_protected.best_estimator_

    text = f"{ratio}:no Adv Training"
    metrics_master(xgb_model, X_test_adv, y_test_adv, text)
    text = f"{ratio}:w/ Adv Training"
    metrics_master(xgb_model_protected, X_test_adv, y_test_adv, text)

Attack Time: 556.8508467674255
0.01:full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.01:full A2PM Attack
Accuracy: 99.99%
Macro Precision: 98.47%
Macro Recall: 93.15%
Macro F1 Score: 95.36%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   3.9s
[CV 2/3] END ..................................., score=1.000 total time=   4.0s
[CV 3/3] END ..................................., score=1.000 total time=   4.5s
Best parameters found:  {}
0.01:no Adv Training
Accuracy: 89.95%
Weighted Precision: 80.91%
Weighted Recall: 89.95%
Weighted F1 Score: 85.19%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.01:no Adv Training
Accuracy: 89.95%
Macro Precision: 74.13%
Macro Recall: 83.33%
Macro F1 Score: 78.46%
0.01:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.01:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%
Attack Time: 551.7298600673676
0.05:full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.05:full A2PM Attack
Accuracy: 99.99%
Macro Precision: 98.77%
Macro Recall: 92.05%
Macro F1 Score: 94.82%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   4.3s
[CV 2/3] END ..................................., score=1.000 total time=   4.1s
[CV 3/3] END ..................................., score=1.000 total time=   4.3s
Best parameters found:  {}
0.05:no Adv Training
Accuracy: 89.95%
Weighted Precision: 80.91%
Weighted Recall: 

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.05:no Adv Training
Accuracy: 89.95%
Macro Precision: 73.49%
Macro Recall: 83.33%
Macro F1 Score: 78.10%
0.05:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.05:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%
Attack Time: 549.7241389751434
0.1:full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.1:full A2PM Attack
Accuracy: 99.99%
Macro Precision: 98.86%
Macro Recall: 93.10%
Macro F1 Score: 95.51%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   4.0s
[CV 2/3] END ..................................., score=1.000 total time=   4.2s
[CV 3/3] END ..................................., score=1.000 total time=   4.2s
Best parameters found:  {}
0.1:no Adv Training
Accuracy: 89.95%
Weighted Precision: 80.91%
Weighted Recall: 89.

...

0.1:no Adv Training
Accuracy: 89.95%
Macro Precision: 75.56%
Macro Recall: 83.33%
Macro F1 Score: 79.24%
0.1:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.1:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 99.56%
Macro Recall: 100.00%
Macro F1 Score: 99.78%
Attack Time: 543.1949362754822
0.2:full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.2:full A2PM Attack
Accuracy: 99.99%
Macro Precision: 99.19%
Macro Recall: 90.48%
Macro F1 Score: 93.92%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   3.9s
[CV 2/3] END ..................................., score=1.000 total time=   4.1s
[CV 3/3] END ..................................., score=1.000 total time=   4.2s
Best parameters found:  {}
0.2:no Adv Training
Accuracy: 89.95%
Weighted Precision: 80.91%
Weighted Recall: 89.95%
W

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.2:no Adv Training
Accuracy: 89.95%
Macro Precision: 72.53%
Macro Recall: 83.33%
Macro F1 Score: 77.51%
0.2:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.2:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 99.17%
Macro Recall: 100.00%
Macro F1 Score: 99.57%
Attack Time: 497.4636039733887
0.3:full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.3:full A2PM Attack
Accuracy: 99.99%
Macro Precision: 97.64%
Macro Recall: 87.97%
Macro F1 Score: 91.76%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   5.9s
[CV 2/3] END ..................................., score=1.000 total time=   4.4s
[CV 3/3] END ..................................., score=1.000 total time=   4.2s
Best parameters found:  {}
0.3:no Adv Training
Accuracy: 89.95%
Weighted Precision: 80.91%
Weighted Recall: 89.95%
W

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3:no Adv Training
Accuracy: 89.95%
Macro Precision: 73.69%
Macro Recall: 83.33%
Macro F1 Score: 78.18%
0.3:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.3:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 99.57%
Macro Recall: 100.00%
Macro F1 Score: 99.78%


In [32]:
xgb = XGBClassifier()
xgb = xgb.fit(X_train, y_train)


In [33]:
X = np.load("/home/jovyan/A2PM/A2PM_adv_sample/A2PM_0.3_fullattack_X.npy")
y = np.load("/home/jovyan/A2PM/A2PM_adv_sample/A2PM_0.3_fullattack_y.npy")
text = f"{0.1}:full A2PM Attack"
metrics_master(xgb, X, y, text)

0.1:full A2PM Attack
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.1:full A2PM Attack
Accuracy: 99.99%
Macro Precision: 98.91%
Macro Recall: 94.52%
Macro F1 Score: 96.46%


In [35]:
X = np.load("/home/jovyan/A2PM/A2PM_adv_sample/A2PM_0.3_10%attack_X.npy")
y = np.load("/home/jovyan/A2PM/A2PM_adv_sample/A2PM_0.3_10%attack_y.npy")
text = f"{0.1}:10% A2PM Attack"
metrics_master(xgb, X, y, text)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.1:10% A2PM Attack
Accuracy: 90.00%
Weighted Precision: 81.00%
Weighted Recall: 90.00%
Weighted F1 Score: 85.26%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.1:10% A2PM Attack
Accuracy: 90.00%
Macro Precision: 75.08%
Macro Recall: 83.33%
Macro F1 Score: 78.99%


In [39]:
np.unique(y)

array([0, 1, 2, 3, 4, 5])

In [41]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

adv_samples_num = int(955009*0.10/0.9) 

X_adversarial = np.load("/home/jovyan/A2PM/A2PM_adv_sample/A2PM_0.3_fullattack_X.npy")

X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
X_adversarial_sampled = X_adversarial[X_adversarial_indices]

y_adv = y[X_adversarial_indices]

X_new = X.copy()
y_new = y.copy()

X_combined = np.vstack((X_new, X_adversarial_sampled))
y_combined = np.concatenate((y_new, y_adv))

metrics_master(xgb, X_combined, y_combined, text)

0.1:10% A2PM Attack
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.1:10% A2PM Attack
Accuracy: 100.00%
Macro Precision: 99.99%
Macro Recall: 99.68%
Macro F1 Score: 99.83%


In [43]:
X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')


ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    xtext = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_X.npy"
    X_adv = np.load(xtext)
    ytext = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_y.npy"
    y_adv = np.load(ytext)
    
    X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_adv, y_adv, test_size=0.2, random_state=42)
    
    
    xgb = XGBClassifier()
    xgb = xgb.fit(X_train, y_train)
    
    text=f"{ratio}_origin"
    metrics_master(xgb, X_test, y_test, text)
    
    text=f"{ratio}_test attack dataset"
    metrics_master(xgb, X_test_adv, y_test_adv, text)
    
    xgb_def = XGBClassifier()
    xgb_def = xgb_def.fit(X_train_adv, y_train_adv)
    text=f"{ratio}_defensed model"
    metrics_master(xgb_def, X_test_adv, y_test_adv, text)

0.01_origin
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.01_origin
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%
0.01_test attack dataset
Accuracy: 99.99%
Weighted Precision: 99.99%
Weighted Recall: 99.99%
Weighted F1 Score: 99.99%
0.01_test attack dataset
Accuracy: 99.99%
Macro Precision: 99.46%
Macro Recall: 95.25%
Macro F1 Score: 97.19%
0.01_defensed model
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.01_defensed model
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%
0.05_origin
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.05_origin
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%
0.05_test attack dataset
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%